In [1]:
import numpy as np
#import astropy?
import matplotlib.pyplot as plt
%matplotlib inline

## Equations of State

The typical equations of state for stars are

$$ \mu_{\text{ions}} = \left(\sum_{i} {\frac{X_{i}}{\mathcal{A}_{i}}}\right)^{-1} $$
$$ \mu_{e} = \left(\sum_{i} {\frac{X_{i} \mathcal{Z}_{i}}{\mathcal{A}_{i}}}\right)^{-1} $$

$$ P = P_{\text{ions}} + P_{e} + P_{\text{rad}} = P_{\text{gas}} + P_{\text{rad}} $$
$$ P_{\text{ions}} = \frac{\rho}{\mu_{\text{ions}} m_{u}} k_{B}T $$
$$ P_{e} = \left\{\begin{array}{ll}
    \frac{\rho}{\mu_{e} m_{u}} k_{B}T && \text{(non-degenerate, non-relativistic)} \\
    \frac{h^{2}}{20 m_{e}} \left(\frac{3}{\pi}\right)^{2/3} \left(\frac{\rho}{\mu_{e} m_{u}}\right)^{5/3} && \text{(degenerate, non-relativistic)} \\
    \frac{hc}{8} \left(\frac{3}{\pi}\right)^{1/3} \left(\frac{\rho}{\mu_{e} m_{u}}\right)^{4/3} && \text{(degenerate, relativistic)}\end{array}\right. $$
$$ P_{\text{rad}} = \frac{4}{3} \frac{\sigma_{SB}}{c} T^{4} $$

$$ \kappa = \left(\frac{1}{\kappa_{\text{rad}}} + \frac{1}{\kappa_{\text{cond}}}\right)^{-1} $$
$$ \kappa_{\text{rad}} = \kappa_{e,\text{rad}} + \kappa_{\text{f-f}} + \kappa_{\text{b-f}} $$
$$ \kappa_{e,\text{rad}} = 0.02 (1+X) $$
$$ \kappa_{\text{f-f}} = 3.68 \cdot 10^{22} \frac{Z^{2}}{\mu_{e} \mu_{\text{ions}}} \rho T^{-7/2} $$
$$ \kappa_{\text{b-f}} = 4.34 \cdot 10^{25} g Z(1+X) \rho T^{-7/2} $$
$$ g = \frac{1}{2.82(\rho (1+X))^{0.2}} $$
$$ \kappa_{\text{cond}} = \kappa_{e,\text{cond}} + \kappa_{\gamma} $$
$$ \kappa_{e,\text{cond}} = \frac{32\pi\sigma_{SB} T^{3}}{3k_{B} \rho} \left(\frac{m_{e}}{3k_{B}T}\right)^{1/2} \left(\frac{\bar{\mathcal{Z}} e^{2}}{4\pi\epsilon_{0} k_{B}T}\right)^{2} $$
$$ \kappa_{\gamma} = \sqrt{3} \bar{\mathcal{Z}} \frac{P_{\text{rad}}}{P_{e}} \left(\frac{m_{e} c^{2}}{k_{B}T}\right)^{5/2} \kappa_{e,\text{cond}} $$

$$ \epsilon \approx \left\{\begin{array}{ll}
    2.4 \cdot 10^{3} \frac{\rho X^{2}}{T^{2/3}} \, \text{exp}\!\left(-3.380 \cdot 10^{3} / T^{1/3}\right) \ \mathrm{W/kg} && \text{($pp$-chains)} \\
    4.4 \cdot 10^{24} \frac{\rho XZ}{T^{2/3}} \, \text{exp}\!\left(-15.288 \cdot 10^{3} / T^{1/3}\right) \ \mathrm{W/kg} && \text{(CNO cycle)} \\
    5.1 \cdot 10^{28} \frac{\rho^{2} Y^{3}}{T^{3}} \, \text{exp}\!\left(-4.4027 \cdot 10^{9} / T\right) \ \mathrm{W/kg} && \text{(triple-$\alpha$)}
\end{array}\right. $$

with variables
* $X_{i}$ - species mass fraction
* $P$ - pressure
* $\rho$ - density
* $\mu$ - mean molecular weight
* $T$ - temperature
* $\kappa$ - opacity
* $\epsilon$ - luminosity per unit mass
* $X$ - hydrogen mass fraction
* $Y$ - helium mass fraction
* $Z$ - metals mass fraction

and constants
* $\mathcal{A}_{i}$ - species atomic mass
* $\mathcal{Z}_{i}$ - species atomic charge
* Nucleon mass $m_{u} = 1.660538782 \cdot 10^{-27} \ \mathrm{kg}$
* Boltzmann constant $k_{B} = 1.3806504 \cdot 10^{-23} \ \mathrm{J/K}$
* Plank's constant $h = 6.6260896 \cdot 10^{-34} \ \mathrm{J \ s}$
* Electron mass $m_{e} = 9.10938215 \cdot 10^{-31} \ \mathrm{kg}$
* Speed of light $c = 299792458 \ \mathrm{m/s}$
* Stefan-Boltzmann constant $\sigma_{SB} = 5.6704 \cdot 10^{-8} \ \mathrm{W/m^{2} \ K^{4}}$
* Electron charge $e = 1.602176487 \cdot 10^{-19} \ \mathrm{C}$
* Electric permittivity of free space $\epsilon_{0} = 8.854187817 \cdot 10^{-12} \ \mathrm{s^{4} \ A^{2}/kg \ m^{3}}$

In [2]:
class DataContainer:
    pass

consts = DataContainer()

consts.A = {
    "H": 1.008, "He": 4.0026, "Li": 6.94, "Be": 9.0122, "B": 10.81, "C": 12.011, "N": 14.007, "O": 15.999, "F": 18.998, 
    "Ne": 20.180, "Na": 22.990, "Mg": 24.305, "Al": 26.982, "Si": 28.085, "P": 30.974, "S": 32.06, "Cl": 35.45, 
    "Ar": 39.948, "K": 39.098, "Ca": 40.078, "Sc": 44.956, "Ti": 47.867, "V": 50.942, "Cr": 51.996, "Mn": 54.938, 
    "Fe": 55.845,
    "Z": 15.5 # Average of solar composition for metals -- catch-all for species with undefined mass fractions
}

consts.Z = {
    "H": 1, "He": 2, "Li": 3, "Be": 4, "B": 5, "C": 6, "N": 7, "O": 8, "F": 9, "Ne": 10, "Na": 11, "Mg": 12, "Al": 13, 
    "Si": 14, "P": 15, "S": 16, "Cl": 17, "Ar": 18, "K": 19, "Ca": 20, "Sc": 21, "Ti": 22, "V": 23, "Cr": 24, "Mn": 25, 
    "Fe": 26,
    "Z": 15.5/2 # Average of solar composition for metals -- catch-all for species with undefined mass fractions
}

consts.m_u = 1.660538782e-27
consts.k_B = 1.3806504e-23
consts.h = 6.6260896e-34
consts.m_e = 9.10938215e-31
consts.c = 2.99792458e8
consts.sigma_SB = 5.6704e-8
consts.e = 1.602176487e-19
consts.epsilon_0 = 8.854187817e-12

In [3]:
def root_finder(f, x0, x1, epsilon=1e-6, **kwargs):
    """
    Finds root of a function in a particular range.
    f - function to find root of
    x0 - lower bound
    x1 - upper bound
    epsilon - error tolerance of root
    **kwargs - additional keyword arguments for f
    """
    
    f0 = f(x0, **kwargs)
    f1 = f(x1, **kwargs)
    
    xmid = (x0+x1)/2
    fmid = f(xmid, **kwargs)
    
    if (fmid == 0):
        return xmid
    else:
        if (fmid > 0):
            x1 = xmid
            f1 = fmid
        else:
            x0 = xmid
            f0 = fmid
    
    root_counter = 1
    while (abs(f1-f0) > epsilon):
        xmid = (x0+x1)/2
        fmid = f(xmid, **kwargs)
        
        if (fmid == 0):
            break
        else:
            if (fmid > 0):
                x1 = xmid
                f1 = fmid
            else:
                x0 = xmid
                f0 = fmid
        
        root_counter += 1    
    
    return xmid

In [4]:
def molecular_weights(X):
    """
    Calculates mean molecular weights for ions (mu_ions) and electrons (mu_e).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    """
    mu_ions = 1/np.sum([X[species]/consts.A[species] for species in X.keys()])
    mu_e = 1/np.sum([X[species]*consts.Z[species]/consts.A[species] for species in X.keys()])
    
    return mu_ions, mu_e

def pressure_ions(X, rho, T):
    """
    Calculates local ion pressure (P_ions).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * rho - local density
    * T - local temperature
    """
    mu_ions, mu_e = molecular_weights(X)
    
    P_ions = rho/(mu_ions*consts.m_u) * consts.k_B*T
    
    return P_ions

def pressure_electrons(X, rho, T):
    """
    Calculates local electron pressure (P_e).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * rho - local density
    * T - local temperature
    """
    mu_ions, mu_e = molecular_weights(X)
    
    rho_trans_degenerate = (20*consts.m_e*consts.k_B)**(3/2) * (np.pi*consts.m_u/(3*consts.h**3)) * mu_e * T**(3/2)
    rho_trans_relativistic = (5/2 * consts.m_e*consts.c/consts.h)**3 * (np.pi*consts.m_u/3) * mu_e
    
    if (rho < rho_trans_degenerate):
        P_e = rho/(mu_e*consts.m_u) * consts.k_B*T
        consts.gamma = 5/3 # Non-relativistic adiabatic index
    elif (rho < rho_trans_relativistic):
        P_e = consts.h**2/(20*consts.m_e) * (3/np.pi)**(2/3) * (rho/(mu_e*consts.m_u))**(5/3)
        consts.gamma = 5/3 # Non-relativistic adiabatic index
    else:
        P_e = consts.h*consts.c/8 * (3/np.pi)*(1/3) * (rho/(mu_e*consts.m_u))**(4/3)
        consts.gamma = 4/3 # Relativistic adiabatic index
    
    return P_e

def pressure_radiation(T):
    """
    Calculates local radiation pressure (P_rad).
    * T - local temperature
    """
    P_rad = 4/3 * consts.sigma_SB/consts.c * T**4
    
    return P_rad

def pressure(X, rho, T):
    """
    Calculates local pressure (P).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * rho - local pressure
    * T - local temperature
    """
    P_ions = pressure_ions(X, rho, T)
    P_e = pressure_electrons(X, rho, T)
    P_rad = pressure_radiation(T)
    
    return P_ions + P_e + P_rad

def pressure_difference(rho, X, P, T):
    """
    Calculates difference between P and pressure(X, rho, T) for density root finding.
    * rho - local density
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * P - local pressure
    * T - local temperature
    """
    return P - pressure(X, rho, T)

def density(X, P, T, rho_surface, rho_central):
    """
    Calculates local density (rho).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * P - local pressure
    * T - local temperature
    * rho_surface - surface density
    * rho_central - central density
    """
    rho = root_finder(pressure_difference, rho_surface, rho_central, X=X, P=P, T=T)
    
    return rho

def reduced_luminosity(X, rho, T):
    """
    Calculates local reduced luminosity (epsilon).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * rho - local density
    * T - local temperature
    """
    epsilon = dict()
    
    if all([species in X.keys() for species in ["H"]]):
        epsilon_pp = 2.4e3 * (rho * X["H"]**2 / T**(2/3)) * np.exp(-3.380e3 / T**(1/3))
        epsilon[r"$pp$-chain"] = epsilon_pp
    if all([species in X.keys() for species in ["H","C","N","O"]]):
        epsilon_CNO = 4.4e24 * (rho*X["H"]*(X["C"]+X["N"]+X["O"]) / T**(2/3)) * np.exp(-15.288e3 / T**(1/3))
        epsilon[r"$\mathrm{CNO}"] = epsilon_CNO
    if all([species in X.keys() for species in ["He"]]):
        epsilon_3alpha = 5.1e28 * (rho**2 * X["He"]**3 / T**3) * np.exp(-4.4027e9 / T)
        epsilon[r"3-$\alpha$"] = epsilon_3alpha
    # Additional energy generation rates for higher-order burning are located in Penn State Lec. 22
    
    return epsilon

def opacity_electrons_radiative(X):
    """
    Calculates local opacity from electron scattering (kappa_e_rad).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    """
    kappa_e_rad = 0.02*(1+X["H"])
    
    return kappa_e_rad

def opacity_free_free(X, rho, T):
    """
    Calculates local opacity from free-free absorption (kappa_ff).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * rho - local density
    * T - local temperature
    """
    mu_ions, mu_e = molecular_weights(X)
    
    kappa_ff = 3.68e22 * ((1-X["H"]-X["He"])**2)/(mu_ions*mu_e) * rho * T**(-7/2)
    
    return kappa_ff

def opacity_bound_free(X, rho, T):
    """
    Calculates local opacity from bound-free absorption (kappa_bf).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * rho - local density
    * T - local temperature
    """
    guillotine = 1/(2.82*(rho*(1+X["H"]))**0.2)
    kappa_bf = 4.34e25 * guillotine * (1-X["H"]-X["He"])*(1+X["H"]) * rho * T**(-7/2)
    
    return kappa_bf

def opacity_radiative(X, rho, T):
    """
    Calculates local radiative opacity (kappa_rad).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * rho - local density
    * T - local temperature
    """
    kappa_e_rad = opacity_electrons_radiative(X)
    kappa_ff = opacity_free_free(X, rho, T)
    kappa_bf = opacity_bound_free(X, rho, T)
    
    return kappa_e_rad + kappa_ff + kappa_bf

def opacity_electrons_conductive(X, rho, T):
    """
    Calculates local opacity from electron conduction (kappa_e_cond).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * rho - local density
    * T - local temperature
    """
    Zbar = np.sum([X[species]*consts.Z[species] for species in X.keys()])
    
    kappa_e_cond = (32*np.pi*consts.sigma_SB * T**3)/(3*consts.k_B * rho) * np.sqrt(consts.m_e/(3*consts.k_B*T)) \
                   * ((Zbar*consts.e**2)/(4*np.pi*consts.epsilon_0*consts.k_B*T))**2
    
    return kappa_e_cond

def opacity_photons_conductive(X, rho, T):
    """
    Calculates local opacity from photon conduction (kappa_gamma).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * rho - local density
    * T - local temperature
    """
    Zbar = np.sum([X[species]*consts.Z[species] for species in X.keys()])
    P_rad = pressure_radiation(T)
    P_e = pressure_electrons(X, rho, T)
    
    kappa_e_cond = opacity_electrons_conductive(X, rho, T)
    kappa_gamma = kappa_e_cond * np.sqrt(3) * Zbar * P_rad/P_e * ((consts.m_e * consts.c**2)/(consts.k_B*T))**(5/2)
    
    return kappa_gamma

def opacity_conductive(X, rho, T):
    """
    Calculates local conductive opacity (kappa_cond).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * rho - local density
    * T - local temperature
    """
    kappa_e_cond = opacity_electrons_conductive(X, rho, T)
    kappa_gamma = opacity_photons_conductive(X, rho, T)
    
    return kappa_e_cond + kappa_gamma

def opacity(X, rho, T):
    """
    Calculates local opacity (kappa).
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * rho - local density
    * T - local temperature
    """
    kappa_rad = opacity_radiative(X, rho, T)
    kappa_cond = opacity_conductive(X, rho, T)
    
    return 1/(1/kappa_rad + 1/kappa_cond)

def temperature_gradient(X, z, rho, kappa, dPdm):
    """
    Calculates temperature gradient based on dominant energy transfer mechanism.
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li:"X_Li, ..., "Z":Z_other]
    * z - array of [r,P,L,T]
    * rho - local density
    * kappa - local opacity
    * dPdm - local pressure gradient
    """
    r, P, L, T = z
    
    mu_ions, mu_e = molecular_weights(X)
    rho_trans_degenerate = (20*consts.m_e*consts.k_B)**(3/2) * (np.pi*consts.m_u/(3*consts.h**3)) * mu_e * T**(3/2)
    
    if (rho < rho_trans_degenerate):
        dPdT = rho*consts.k_B/consts.m_u * (1/mu_ions + 1/mu_e) + 16/3 * consts.sigma_SB/consts.c * T**3
    else:
        dPdT = rho*consts.k_B/(mu_ions*consts.m_u) + 16/3 * consts.sigma_SB/consts.c * T**3
    
    if (P/T / dPdT) < (1-1/consts.gamma): # radiative/conductive
        dTdm = -3*kappa*L/(256*np.pi**2 * r**4 * consts.sigma_SB * T**3)
    else:
        dTdm = (1-1/consts.gamma) * T/P * dPdm
    
    return dTdm

## Stellar Structure

The time-independent equations of stellar structure are

$$ \frac{\partial r(m)}{\partial m} = \frac{1}{4\pi r^{2}(m) \, \rho(m)} $$

$$ \frac{\partial P(m)}{\partial m} = -\frac{Gm}{4\pi r^{4}(m)} $$

$$ \frac{\partial L(m)}{\partial m} = \epsilon(m) $$

$$ \frac{\partial T(m)}{\partial m} = \left\{\begin{array}{ll}
    -\frac{3\kappa L(m)}{256\pi^{2} r^{4}(m) \, \sigma_{SB} T^{3}(m)} && \text{(radiative/conductive)} \\
    \left(1 - \frac{1}{\gamma}\right) \frac{T(m)}{P(m)} \frac{\partial P(m)}{\partial m} && \text{(convective)}
\end{array}\right. $$

with variables
* $r$ - radius
* $M$ - mass
* $\rho$ - density
* $P$ - pressure
* $L$ - luminosity
* $\epsilon$ - luminosity per unit mass
* $T$ - temperature
* $\kappa$ - opacity

and constants
* Gravitational constant $G = 6.673 \cdot 10^{-11} \ \mathrm{m^{3}/kg \ s^{2}}$
* Stefan-Boltzmann constant $\sigma_{SB} = 5.6704 \cdot 10^{-8} \ \mathrm{W/m^{2} \ K^{4}}$
* Adiabatic index $\gamma = \left\{\frac{5}{3}, \text{(relativtistic)}; \frac{4}{3}, \text{(relativistic)}\right\}$

In [5]:
consts.G = 6.673e-11
consts.gamma = 5/3 # Assume non-relativistic adiabatic index to start

In [6]:
def stellar_derivatives(z, m, STAR):
    """
    Calculates stellar derivatives (dz/dm).
    * z - array of current values of [r, P, L, T]
    * m - current mass value
    * STAR - Star object containing mass MSTAR, mass fractions X, and central/surface boundary conditions
    """
    r, P, L, T = z
    
    rho = density(STAR.X, P, T, STAR.rho_surface, STAR.rho_central)
    epsilon = reduced_luminosity(STAR.X, rho, T)
    kappa_rad = opacity_radiative(STAR.X, rho, T)
    kappa_cond = opacity_conductive(STAR.X, rho, T)
    kappa = opacity(STAR.X, rho, T)
    
    drdm = 1/(4*np.pi * r**2 * rho)
    dPdm = -1/(4*np.pi * r**2) * (consts.G*m/r**2)
    dLdm = np.sum(list(epsilon.values()))
    dTdm = temperature_gradient(STAR.X, z, rho, kappa, dPdm)
    
    dzdm = np.array([drdm, dPdm, dLdm, dTdm])
    
    return dzdm

## Boundary Conditions

Outer:

* $M = \underset{m \to M}{\text{lim}} m$
* $L_{\text{s}} = \underset{m \to M}{\text{lim}} \ L(m) = \left(\frac{M}{M_{\odot}}\right)^{3.5} L_{\odot}$ ([Citation](https://en.wikipedia.org/wiki/Mass%E2%80%93luminosity_relation))
* $R = \underset{m \to M}{\text{lim}} \ r(m) = \left\{\begin{array}{ll}
    \left(\frac{M}{M_{\odot}}\right)^{0.8} R_{\odot} & M \le M_{\odot} \\
    \left(\frac{M}{M_{\odot}}\right)^{0.57} R_{\odot} & M > M_{\odot}
\end{array}\right.$
* $T_{\text{s}} = \underset{m \to M}{\text{lim}} \ T(m) = \left(\frac{L_{\text{s}}}{4\pi\sigma_{SB} R^{2}}\right)^{1/4}$
* $\rho_{\text{s}} = \underset{m \to M}{\text{lim}} \ \rho(m) \approx \sqrt{\frac{2}{3}} \left[\rho\!\left(\vec{X}, \kappa_{\text{f-f}}, T_{\text{s}}\right) + \rho\!\left(\vec{X}, \kappa_{\text{b-f}}, T_{\text{s}}\right)\right]^{-1/2}$
* $\kappa_{\text{s}} = \underset{m \to M}{\text{lim}} \ \kappa(m) = \kappa_{\text{f-f}}\!\left(\vec{X}, \rho_{\text{s}}, T_{\text{s}}\right) + \kappa_{\text{b-f}}\!\left(\vec{X}, \rho_{\text{s}}, T_{\text{s}}\right)$
* $P_{\text{s}} = \underset{m \to M}{\text{lim}} \ P(m) = \frac{2}{3} \frac{GM}{\kappa_{\text{s}} R^{2}} + \frac{2}{3c} \frac{L_{\text{s}}}{4\pi R^{2}}$

Inner:

* $M_{0} = \underset{m \to 0}{\text{lim}} \ m \implies \frac{M_{0}}{M} \approx 0$
* $\rho_{\text{c}} = \underset{m \to M_{0}}{\text{lim}} \ \rho(m) \propto \frac{3}{4\pi} \frac{M}{R^{3}}$
* $R_{\text{c}} = \underset{m \to M_{0}}{\text{lim}} \ r(m) = \left(\frac{3}{4\pi} \frac{M_{0}}{\rho_{\text{c}}}\right)^{1/3}$
* $P_{\text{c}} = \underset{m \to M_{0}}{\text{lim}} \ P(m) \propto \frac{GM^{2}}{R^{4}}$
* $T_{\text{c}} = \underset{m \to M_{0}}{\text{lim}} \ T(m) = \frac{2}{3} \frac{G}{k_{B}} \frac{M m_{u}}{R}$
* $L_{\text{c}} = \underset{m \to M_{0}}{\text{lim}} \ L(m) = M_{0} \epsilon\!\left(\vec{X}, \rho_{\text{c}}, T_{\text{c}}\right)$

with constants
* Solar mass $M_{\odot} = 1.9891 \cdot 10^{30} \ \mathrm{kg}$
* Solar luminosity $L_{\odot} = 3.84 \cdot 10^{26} \ \mathrm{W}$
* Solar radius $R_{\odot} = 6.95508 \cdot 10^{8} \ \mathrm{m}$

In [7]:
consts.M_sun = 1.9891e30
consts.R_sun = 6.95508e8
consts.L_sun = 3.84e26
consts.a_0 = 5.29177210903e-11

In [8]:
def estimate_boundary_vals(STAR, perturbations):
    """
    Estimates values of integration variables at stellar center and surface
    * STAR - Star object containing mass MSTAR and mass fractions X
    * perturbations - array of perturbations to [R,P_central,L_surf,T_central]
    """
    mu_ions, mu_e = molecular_weights(STAR.X)
    
    # Surface values
    L_surface = (STAR.M/consts.M_sun)**3.5 * consts.L_sun + perturbations[2]
    if (STAR.M <= consts.M_sun):
        R = (STAR.M/consts.M_sun)**0.8 * consts.R_sun + perturbations[0]
    else:
        R = (STAR.M/consts.M_sun)**0.57 * consts.R_sun + perturbations[0]
    T_surface = (L_surface/(4*np.pi*consts.sigma_SB * R**2))**(1/4)
    rho_surface = np.sqrt(2/3)/np.sqrt(3.68e22 * ((1-STAR.X["H"]-STAR.X["He"])**2)/(mu_ions*mu_e) * T_surface**(-7/2) + 
                                       4.34e25 * (1-STAR.X["H"]-STAR.X["He"]) * (1+STAR.X["H"]) * T_surface**(-7/2))
    kappa_surface = opacity_free_free(STAR.X, rho_surface, T_surface) + opacity_bound_free(STAR.X, rho_surface, T_surface)
    P_surface = 2/3 * (consts.G*STAR.M)/(kappa_surface*R**2) + 2/(3*consts.c) * L_surface/(4*np.pi*R**2)
    
    # Central values
    #R_central = consts.a_0 # Must be non-zero for stellar derivatives to function properly
    #L_central = 0
    #rho_central = 5.9905 * 3/(4*np.pi) * (STAR.M/R**3)
    rho_central = 115.0 * 3/(4*np.pi) * (STAR.M/R**3)
    R_central = (3/(4*np.pi) * (STAR.M*1e-10)/rho_central)**(1/3)
    #P_central = 0.7701*consts.G * (STAR.M**2)/R**4 + perturbations[1]
    P_central = 7.701*consts.G * (STAR.M**2)/R**4 + perturbations[1]
    T_central = 2/3 * consts.G/consts.k_B * STAR.M*consts.m_u/R + perturbations[3]
    # Recalculate density subject to perturbations
    #rho_central = consts.m_e/(consts.k_B*(1/mu_ions + 1/mu_e)) * P_central/T_central
    epsilon = reduced_luminosity(STAR.X, rho_central, T_central)
    L_central = np.sum(list(epsilon.values())) * STAR.M*1e-10
    
    return R_central, P_central, L_central, T_central, rho_central, R, P_surface, L_surface, T_surface, rho_surface

## Computational Process

The process we will undertake is to specify boundary conditions at both the center and the surface, then simultaneously integrate both inwards and outwards until the solutions reach the same mass coordinate. Then, we can calculate the solution residuals $\vec{\mathcal{R}} = \vec{z}_{\text{outward}} - \vec{z}_{\text{inward}}$. Then, we can calculate an update to our initial conditions $\vec{z}_{0} = [R_{\text{central}},P_{\text{central}},L_{\text{central}},T_{\text{central}},R,P_{\text{surface}},L_{\text{total}},T_{\text{effective}}]$ by solving

$$ \sum_{j} {\frac{\partial\mathcal{R}_{i}}{\partial (z_{0})_{j}} \delta (z_{0})_{j}} = -\mathcal{R}_{i} $$

for the perturbations $\delta (z_{0})_{j}$. Note that each partial derivative will require another set of inward and outward integration.

1. Specify total mass $M$ and chemical composition $\vec{X} = \left[X,Y,Z\right]$  
2. Specify internal boundary conditions (estimate for some $m = M_{0} \ll M$) and surface boundary conditions ($m = M$)  
3. Initialize variables $\vec{z}_{\text{outward}}$ and $\vec{z}_{\text{inward}}$  
4. Calculate additional quantities (e.g., $\rho(m)$) using equations of state  
5. Calculate derivatives $\frac{\partial\vec{z}_{\text{outward}}}{\partial m}$ and $\frac{\partial\vec{z}_{\text{inward}}}{\partial m}$  
6. Integrate both solutions using 4th-order Runge-Kutta method  
  a. Stepsize $h$ should be of order $\text{min}\!\left(\frac{\partial\ln(\vec{z})}{\partial m}\right)$ in both directions  
7. Repeat steps 4-6 until reaching $m_{\text{outward}} = m_{\text{inward}}$
8. Calulate differences between solutions at shell midpoint (residuals $\vec{\mathcal{R}} = \vec{z}_{\text{outward}} - \vec{z}_{\text{inward}}$)
11. Calculate perturbations using residuals to update guesses for boundary values
12. Repeat steps 2-11 until inner/outer solutions match to within a certain tolerance

In [9]:
def clamp(val, low, high):
    """
    Returns value clamped between [low,high].
    * val - value to be clamped
    * low - lower bound
    * high - upper bound
    """
    return max(low, min(high, val))

def rk4_integration(f, r, t, h, **kwargs):
    """
    Numeric ODE integration via the 4th-order Runge-Kutta method.
    * f - function that computes dr/dr
    * r - quantity (or vector of quantities) being integrated
    * t - independent integration variable
    * h - stepsize for integration variable
    * **kwargs - additional keyword arguments for f
    """
    
    k1 = h*f(r, t, **kwargs)
    k2 = h*f(r+k1/2, t+h/2, **kwargs)
    k3 = h*f(r+k2/2, t+h/2, **kwargs)
    k4 = h*f(r+k3, t+h, **kwargs)
    
    dr = (k1 + 2*k2 + 2*k3 + k4)/6
    
    if np.isinf(dr).any() or np.isnan(dr).any():
        raise Exception("Cannot update with NaN or inf values")
    
    return r + (k1 + 2*k2 + 2*k3 + k4)/6

In [10]:
def single_stellar_integrator(STAR, perturbations, ODE_integrator, xi, max_steps):
    """
    Integrates equations of stellar structure for a star of a given mass.
    * STAR - Star object containing mass MSTAR and mass fractions X
    * perturbations - array of perturbations to [R,P_central,L_surf,T_central]
    * ODE_integrator - function to integrate PDEs of stellar structure
    * xi - multiplicative fraction for integration stepsize
    * max_steps - maximum number of iterations allowed for convergence
    """
    boundaries = estimate_boundary_vals(STAR, perturbations)
    print("Central boundary conditions:", boundaries[:4])
    print("Surface boundary conditions:", boundaries[5:-1])
    assert (np.array(boundaries) >= 0).all()
    
    #m1 = consts.m_u
    m1 = STAR.M * 1e-10
    z1_0 = np.array(boundaries[:4]) # [R_central, P_central, L_central, and T_central]
    rho_central = boundaries[4]
    m2 = STAR.M
    z2_0 = np.array(boundaries[5:-1]) # [R, P_surface, L_total, and T_effective]
    rho_surface = boundaries[-1]
    
    STAR.boundaries_central = boundaries[:4]
    STAR.rho_central = boundaries[4]
    STAR.boundaries_surface = boundaries[5:-1]
    STAR.rho_surface = boundaries[9]
    
    m1s = [m1]
    m2s = [m2]
    z1s = [z1_0]
    z2s = [z2_0]
    z1 = np.copy(z1_0)
    z2 = np.copy(z2_0)
        
    Nsteps = 0
    meeting = False
    
    for i in range(1,max_steps):
        rk41 = False
        rk42 = False
        
        r1, P1, L1, T1 = z1
        r2, P2, L2, T2 = z2
        
        dz1dm = stellar_derivatives(z1, m1, STAR)
        dz2dm = stellar_derivatives(z2, m2, STAR)
        hz1 = np.abs(z1/dz1dm)
        hz2 = np.abs(z2/dz2dm)
        #h1 = max(1e-10, min(1e-4, xi*np.min(hz1))) # Smallest of z/(dz/dm) in [1e-10,1e-4]
        #h2 = max(1e-10, min(1e-4, xi*np.min(hz2)))
        h1 = min(m1*1e-2, xi*np.min(hz1))
        h2 = min(m2*1e-2, xi*np.min(hz2))
        if (m1+h1 > m2-h2):
            m_mid = ((m1+h1)+(m2-h2))/2
            h1 = m_mid-m1
            h2 = m2-m_mid
            meeting = True
        
        while (rk41 == False):
            try:
                z1 = rk4_integration(stellar_derivatives, z1, m1, h1, STAR=STAR)
                rk41 = True
            except:
                h1 *= 0.1
        while (rk42 == False):
            try:
                z2 = rk4_integration(stellar_derivatives, z2, m2, h2, STAR=STAR)
                rk42 = True
            except:
                h2 *= 0.1
        
        m1 += h1
        m2 -= h2
        
        if np.isinf(z1).any() or np.isnan(z1).any():
            print("Step:", i)
            print("m1:", m1)
            print("z1:", z1)
            print("Previous z1:", z1s[-1])
            print("dz1dm:", dz1dm)
            print("hz1:", hz1)
            print("h1:", h1)
            raise Exception("Variable cannot take NaN or inf value")
        if np.isinf(z2).any() or np.isnan(z2).any():
            print("Step:", i)
            print("m2:", m2)
            print("z2:", z2)
            print("Previous z2:", z2s[-1])
            print("dz2dm:", dz2dm)
            print("hz2:", hz2)
            print("h2:", h2)
            raise Exception("Variable cannot take NaN or inf value")
        
        m1s.append(m1)
        m2s.append(m2)
        z1s.append(z1)
        z2s.append(z2)
    
        Nsteps += 1
        
        if (Nsteps % 1000 == 0):
            print("Completed %i steps"%Nsteps)
        
        if (meeting == True):
            print("Step:", i)
            print("m1:", m1)
            print("z1:", z1)
            print("m2:", m2)
            print("z2:", z2)
            break
    else: # Else on for loop -- activates if for loop exits normally
        print("m1:", m1)
        print("z1:", z1)
        print("m2:", m2)
        print("z2:", z2)
        raise Exception("Maximum number of iterations reached without convergence")

    return m1s, z1s, m2s, z2s
            
def stellar_perturbations(STAR, residuals, perturbations, ODE_integrator, xi, max_steps):
    """
    Calculates perturbations to boundary conditions using residuals between inward and outward solutions.
    * STAR - Star object containing mass MSTAR and mass fractions X
    * residuals - residuals between outward and inward solutions at midpoint
    * perturbations - array of initial perturbations to [R,P_central,L_surf,T_central]
    * ODE_integrator - function to integrate PDEs of stellar structure
    * xi - multiplicative fraction for integration stepsize
    * max_steps - maximum number of iterations allowed for convergence
    """
    residuals_matrix = np.zeros((4,4))
    
    boundaries = estimate_boundary_vals(STAR, perturbations)
    bcs = np.array([boundaries[5], boundaries[1], boundaries[7], boundaries[3]])
    
    for i, pert in enumerate(perturbations):
        new_perturbations = np.copy(perturbations)
        if (pert == 0):
            new_perturbations[i] += 1e-2 * bcs[i]
        else:
            new_perturbations[i] *= 1.01
        m1s_pert, z1s_pert, m2s_pert, z2s_pert = single_stellar_integrator(STAR, new_perturbations, 
                                                                           ODE_integrator, xi, max_steps)
        residuals_pert = z2s_pert[-1] - z1s_pert[-1]
        residuals_matrix[:,i] = (residuals_pert-residuals)/(new_perturbations[i]-perturbations[i])
    
    print(residuals_matrix)
    try:
        updated_perturbations = np.linalg.solve(residuals_matrix, -1*residuals)
    except:
        updated_perturbations = np.linalg.lstsq(residuals_matrix, -1*residuals)
    
    try:
        assert (np.abs(updated_perturbations/bcs) < 1).all()
    except:
        updated_perturbations = np.array([clamp(frac_pert, -0.2, 0.2) for frac_pert in updated_perturbations/bcs])*bcs
    #updated_perturbations /= np.abs(perturbations)
    #updated_perturbations = np.array([clamp(pert, -1.1, 1.1) for pert in updated_perturbations])
    
    return updated_perturbations

def full_stellar_integrator(MSTAR, X, ODE_integrator=rk4_integration, xi=0.1, max_steps=10000, delta=1e-3):
    """
    Fully integrates equations of stellar structure with solution matching in the center.
    * MSTAR - total stellar mass
    * X - dictionary of mass fractions ["H":X_H, "He":X_He, "Li":X_Li, ..., "Z":Z_other]
    * ODE_integrator - function to integrate PDFs of stellar structure
    * xi - multiplicative fraction for integration stepsize
    * max_steps - maximum number of iterations allowed for convergence
    * delta - error tolerance for residuals
    """
    assert np.sum(list(X.values())) <= 1
    if "Z" not in X.keys():
        X["Z"] = 1-np.sum(list(X.values()))
    
    STAR = DataContainer()
    STAR.M = MSTAR
    STAR.X = X
    
    perturbations = np.zeros(4)
    
    m1s, z1s, m2s, z2s = single_stellar_integrator(STAR, perturbations, ODE_integrator, xi, max_steps)
    print("r1s:", np.array([zed[0] for zed in z1s])[0:50])
    residuals = z2s[-1] - z1s[-1]
    print("Residuals:", residuals)
    
    new_perturbations = stellar_perturbations(STAR, residuals, perturbations, ODE_integrator, xi, max_steps)
    print("Perturbations:", new_perturbations)
    
    orig_boundaries = estimate_boundary_vals(STAR, perturbations)
    orig_bcs = np.array([orig_boundaries[5], orig_boundaries[1], orig_boundaries[7], orig_boundaries[3]])
    
    residuals_counter = 1
    while (np.abs(residuals/orig_bcs) > delta).any():
        print("Fractional residuals:", np.abs(residuals/orig_bcs))
        print("Residuals counter:", residuals_counter)
        perturbations = np.copy(new_perturbations)
        
        m1s, z1s, m2s, z2s = single_stellar_integrator(STAR, perturbations, ODE_integrator, xi, max_steps)
        print("r1s:", np.array([zed[0] for zed in z1s])[0:50])
        residuals = z2s[-1] - z1s[-1]
        print("Residuals:", residuals)
        new_perturbations = stellar_perturbations(STAR, residuals, perturbations, ODE_integrator, xi, max_steps)
        
        residuals_counter += 1
    
    m1s, z1s, m2s, z2s = single_stellar_integrator(STAR, perturbations, ODE_integrator, xi, max_steps)
    
    r1s = np.array([zed[0] for zed in z1s])
    P1s = np.array([zed[1] for zed in z1s])
    L1s = np.array([zed[2] for zed in z1s])
    T1s = np.array([zed[3] for zed in z1s])
    
    r2s = np.array([zed[0] for zed in z2s])
    P2s = np.array([zed[1] for zed in z2s])
    L2s = np.array([zed[2] for zed in z2s])
    T2s = np.array([zed[3] for zed in z2s])

    #ms = np.concatenate((m1s[:-1],np.array([np.mean(m1s[-1],m2s[-1])]),m2s[::-1][1:]))
    #rs = np.concatenate((r1s[:-1],np.array([np.mean(r1s[-1],r2s[-1])]),r2s[::-1][1:]))
    #Ps = np.concatenate((P1s[:-1],np.array([np.mean(P1s[-1],P2s[-1])]),P2s[::-1][1:]))
    #Ls = np.concatenate((L1s[:-1],np.array([np.mean(L1s[-1],L2s[-1])]),L2s[::-1][1:]))
    #Ts = np.concatenate((T1s[:-1],np.array([np.mean(T1s[-1],T2s[-1])]),T2s[::-1][1:]))
    
    #return ms, rs, Ps, Ls, Ts
    return m1s, r1s, P1s, L1s, T1s, m2s, r2s, P2s, L2s, T2s

In [11]:
M = consts.M_sun
X = {
    "H": 0.7346, "He": 0.2485, "C": 0.0029, "N": 0.0009, "O": 0.0077, "Ne": 0.0012, "Mg": 0.0005, "Si": 0.0007, 
    "S": 0.0004, "Fe": 0.0016
}

#ms, rs, Ps, Ls, Ts = full_stellar_integrator(M, X)
m1s, r1s, P1s, L1s, T1s, m2s, r2s, P2s, L2s, T2s = full_stellar_integrator(M, X)

Central boundary conditions: (66384.93224395563, 8689063356644899.0, 8.646429005661164e+17, 15302051.292250948)
Surface boundary conditions: (695508000.0, 0.14051796466618688, 3.84e+26, 5777.312389262587)
Completed 1000 steps


C:\Users\Brandon Pries\AppData\Local\Temp\ipykernel_3604\566770155.py:33: RuntimeWarning: invalid value encountered in double_scalars
  rho_trans_degenerate = (20*consts.m_e*consts.k_B)**(3/2) * (np.pi*consts.m_u/(3*consts.h**3)) * mu_e * T**(3/2)
C:\Users\Brandon Pries\AppData\Local\Temp\ipykernel_3604\566770155.py:103: RuntimeWarning: invalid value encountered in double_scalars
  epsilon_pp = 2.4e3 * (rho * X["H"]**2 / T**(2/3)) * np.exp(-3.380e3 / T**(1/3))
C:\Users\Brandon Pries\AppData\Local\Temp\ipykernel_3604\566770155.py:106: RuntimeWarning: invalid value encountered in double_scalars
  epsilon_CNO = 4.4e24 * (rho*X["H"]*(X["C"]+X["N"]+X["O"]) / T**(2/3)) * np.exp(-15.288e3 / T**(1/3))
C:\Users\Brandon Pries\AppData\Local\Temp\ipykernel_3604\566770155.py:133: RuntimeWarning: invalid value encountered in double_scalars
  kappa_ff = 3.68e22 * ((1-X["H"]-X["He"])**2)/(mu_ions*mu_e) * rho * T**(-7/2)
C:\Users\Brandon Pries\AppData\Local\Temp\ipykernel_3604\566770155.py:145: Runtime

Completed 2000 steps
Step: 2318
m1: 1.9923002945807883e+30
z1: [1.57287932e+13 7.75449199e+15 2.73307761e+26 7.15832894e+06]
m2: 1.9891000320029433e+30
z2: [6.95508000e+08 1.44460566e+06 3.82008149e+26 1.78273528e+07]
r1s: [66384.93224396 66605.4818118  66826.76410784 67048.78156641
 67271.53662993 67495.03174893 67719.26938209 67944.25199624
 68169.98206643 68396.46207592 68623.69451622 68851.68188711
 69080.42669669 69309.93146139 69540.19870599 69771.23096367
 70003.03077602 70235.60069306 70468.94327332 70703.06108379
 70937.95670001 71173.63270606 71410.09169464 71647.33626702
 71885.36903314 72124.1926116  72363.8096297  72604.22272348
 72845.43453772 73087.44772601 73330.26495073 73573.88888311
 73818.32220328 74063.56760024 74309.62777195 74556.50542532
 74804.20327626 75052.72404968 75302.07047959 75552.24530903
 75803.25129019 76055.09118439 76307.76776213 76561.2838031
 76815.64209625 77070.84543976 77326.89664115 77583.79851722
 77841.55389417 78100.16560757]
Residuals: [-1

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Brandon Pries\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Brandon Pries\AppData\Local\Temp\ipykernel_3604\2711309822.py", line 8, in <module>
    m1s, r1s, P1s, L1s, T1s, m2s, r2s, P2s, L2s, T2s = full_stellar_integrator(M, X)
  File "C:\Users\Brandon Pries\AppData\Local\Temp\ipykernel_3604\3797495146.py", line 186, in full_stellar_integrator
    new_perturbations = stellar_perturbations(STAR, residuals, perturbations, ODE_integrator, xi, max_steps)
  File "C:\Users\Brandon Pries\AppData\Local\Temp\ipykernel_3604\3797495146.py", line 141, in stellar_perturbations
    m1s_pert, z1s_pert, m2s_pert, z2s_pert = single_stellar_integrator(STAR, new_perturbations,
  File "C:\Users\Brandon Pries\AppData\Local\Temp\ipykernel_3604\3797495146.py", line 45, in single_stellar_integrator
    dz1dm = stellar_derivatives(z1, m1, STAR)
  Fi

TypeError: object of type 'NoneType' has no len()

In [ ]:
ms = np.concatenate((m1s[:-1],np.array([np.mean([m1s[-1],m2s[-1]])]),m2s[::-1][1:]))
rs = np.concatenate((r1s[:-1],np.array([np.mean([r1s[-1],r2s[-1]])]),r2s[::-1][1:]))
Ps = np.concatenate((P1s[:-1],np.array([np.mean([P1s[-1],P2s[-1]])]),P2s[::-1][1:]))
Ls = np.concatenate((L1s[:-1],np.array([np.mean([L1s[-1],L2s[-1]])]),L2s[::-1][1:]))
Ts = np.concatenate((T1s[:-1],np.array([np.mean([T1s[-1],T2s[-1]])]),T2s[::-1][1:]))

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.plot(ms)
plt.xlabel("Step")
plt.ylabel(r"$m$ $[\mathrm{kg}]$")

fig, ax = plt.subplots(2,2, figsize=(12,12))

ax[0][0].plot(rs)
ax[0][0].set_xlabel("Step")
ax[0][0].set_ylabel(r"$r(m)$ $[\mathrm{m}]$")

ax[0][1].plot(Ps)
ax[0][1].set_xlabel("Step")
ax[0][1].set_ylabel(r"$P(m)$ $[\mathrm{N/m^{2}}]$")

ax[1][0].plot(Ls)
ax[1][0].set_xlabel("Step")
ax[1][0].set_ylabel(r"$L(m)$ $[\mathrm{W}]$")

ax[1][1].plot(Ts)
ax[1][1].set_xlabel("Step")
ax[1][1].set_ylabel(r"$T(m)$ $[\mathrm{K}]$")

Resources consulted:

**Code:**
* [ZAMS GitHub code](https://github.com/MSU-AST304-FS2020/gcp3-main-sequence-aspen/)
* [STATSTAR code](http://spiff.rit.edu/classes/phys370/lectures/statstar/statstar_python3.py)

**Textbooks:**
* [Prialnik "An Introduction to the Theory of Stellar Structure and Evolution"](https://icourse.club/uploads/files/fa6ee05bde8e18abf27d7e7e444eb7a5e5c47bc7.pdf)
* [Hansen, Kawaler, Trimble "Stellar Interiors (Physical Principles, Structure, and Evolution)"](https://ia600207.us.archive.org/7/items/springer_10.1007-978-1-4419-9110-2/10.1007-978-1-4419-9110-2.pdf)
* [Carroll, Ostlie "An Introduction to Modern Astrophysics](https://www.academia.edu/42881683/An_Introduction_to_Modern_Astrophysics)
* [Novotny "Introduction to Stellar Atmospheres and Interiors"](https://ui.adsabs.harvard.edu/abs/1973itsa.book.....N/abstract)
* [Oswalt, Barstow "Planets, Stars and Stellar Systems"](https://link.springer.com/referenceworkentry/10.1007/978-94-007-5615-1_11)

**Journal articles:**
* [Ekström "Massive Star Modeling and Nucleosynthesis" (FASS)](https://www.frontiersin.org/articles/10.3389/fspas.2021.617765/full)
* [Cowling "The Development of the Theory of Stellar Structure" (QJRAS)](https://articles.adsabs.harvard.edu/pdf/1966QJRAS...7..121C)

**Lecture slides/notes:**
* [Penn State lectures](http://personal.psu.edu/rbc3/A534/)
* [RIT lectures](http://spiff.rit.edu/classes/phys370/lectures/)
* [Radbound University](https://www.astro.ru.nl/~onnop/education/stev_utrecht_notes/)
* [Princeton stellar structure](https://www.astro.princeton.edu/~gk/A403/stellar.pdf)
* [Oxford stellar structure](http://www-astro.physics.ox.ac.uk/~aelg/Krakow/L04.pdf)
* [Ohio State stellar structure](https://www.astronomy.ohio-state.edu/weinberg.21/Intro/lec6.html)
* [UNAM equations of state](https://www.irya.unam.mx/gente/j.arthur/ESTELAR/pols-chapter3-4-new.pdf)
* [St. Andrews stellar opacity](http://www-star.st-and.ac.uk/~kw25/teaching/stars/STRUC7.pdf)
* [Princeton opacity](https://www.astro.princeton.edu/~gk/A403/opac.pdf)
* [Case Western Reserve stellar interiors](http://burro.case.edu/Academics/Astr221/StarPhys/stellarint.html)
* [New Mexico State mean molecular weight](http://astronomy.nmsu.edu/jasonj/565/docs/09_03.pdf)
* [ETH Zurich stellar scaling relations](https://ethz.ch/content/dam/ethz/special-interest/phys/particle-physics/quanz-group-dam/documents-old-s-and-p/Courses/AstrophysikIHS2017/slides_week5_v1.pdf)
* [University College London stellar classification](http://www.star.ucl.ac.uk/~pac/spectral_classification.html)
* [Binghamton relativistic stat mech](https://bingweb.binghamton.edu/~suzuki/ThermoStat.html)

**Wikipedia articles:**
* [Wikipedia stellar structure](https://en.wikipedia.org/wiki/Stellar_structure)
* [Wikipedia thermal conductivity](https://en.wikipedia.org/wiki/Thermal_conductivity)
* [Wikipedia list of thermal conductivities](https://en.wikipedia.org/wiki/List_of_thermal_conductivities)
* [Wikipedia thermal conduction](https://en.wikipedia.org/wiki/Thermal_conduction)
* [Wikipedia heat equation](https://en.wikipedia.org/wiki/Heat_equation)
* [Wikipedia solar core](https://en.wikipedia.org/wiki/Solar_core)
* [Wikipedia stellar core](https://en.wikipedia.org/wiki/Stellar_core)
* [Wikipedia Hertzsprung-Russell diagram](https://en.wikipedia.org/wiki/Hertzsprung%E2%80%93Russell_diagram)
* [Wikipedia Planck's law](https://en.wikipedia.org/wiki/Planck%27s_law)
* [Wikipedia ultraviolet catastrophe](https://en.wikipedia.org/wiki/Ultraviolet_catastrophe)
* [Wikipedia Sun](https://en.wikipedia.org/wiki/Sun)